# GAP Habitat Models as Species-Level Dictionaries
A script for developing a workflow to get model dictionaries onto ScienceBase

In [2]:
import gapproduction as gp
import pysb
from IPython.display import display

In [3]:
# Where to save json files
outDir = "P:/Proj3/USGap/Vert/Model/2001/GAP_habitat_models/"

In [13]:
# Set up your ScienceBase session for operating against. 
sb = pysb.SbSession()
username = input("Username: ")
sb.loginc(str(username))

Username: "nmtarr@ncsu.edu"
········


<pysb.SbSession.SbSession instance at 0x0000000012671A08>

Connect to ScienceBase and get a list of gap species codes from the list of child IDs for the habitat ma ID.

In [18]:
# This is the top level ScienceBase Item ID for the habitat maps
_gapHabitatMapCollectionItem = "527d0a83e4b0850ea0518326"

# This gets all the child IDs from the habitat map collection; returns a list
# you can loop through
habitatMapIDs = sb.get_child_ids(_gapHabitatMapCollectionItem)

# Get list of strUCs from habitatMapIDs to loop on
spUCs = [sb.get_item(x)["identifiers"][0]["key"] for x in habitatMapIDs[:2]]
print(spUCs)

[u'rRNSNx', u'bEASOx']


# Draft dictionary
The next cell creates the dictionaries and saves them as json files.
We'll need to make sure the format is good for everyone and fill in some blanks.  The keys are currently 'models', 'taxonomic', 'doi', 'input_layers', 'habitat_description', and 'references'

# What should the doi be for each species?
# What are the url's or other code-readable locations of the ancillary data layers (elevation etc.)?

# I still need to figure out how to assess hydro selections with code that will identify the right hydro layer for the species to list in "input_layers".


In [25]:
# Generate species model dictionaries and save as json files.
counter=0
for species in spUCs:
    counter += 1
    print(species)
    print(str(counter) + " of " + str(len(spUCs)))
    
    # Make an empty dictionary to collect model dictionaries
    speciesDict = {"input_layers":gp.gapmodeling.layers_2001}
    
    # List taxanomic information
    taxonomic = {"common_name":gp.gapdb.NameCommon(species), 
                 "scientific_name":gp.gapdb.NameSci(species),
                 "gap_code": gp.gapdb.Crosswalk(species)[0],
                 "ELCode": gp.gapdb.Crosswalk(species)[1],
                 "ITIS_TSN": gp.gapdb.Crosswalk(species)[2],
                 "Global_SEQ_ID": gp.gapdb.Crosswalk(species)[3]}
    speciesDict["taxonomic"] = taxonomic
    
    # Add placeholder for DOI
    speciesDict["doi"] = "???????"
    
    # Add the species' habitat description
    description = gp.gapmodeling.getHabitatDescription(species)
    speciesDict["habitat_description"] = description
    
    # Add species' references/citations
    referencesDF = gp.gapmodeling.SpReferences(species)
    references = dict(referencesDF["memCitation"])
    speciesDict["references"] = references
    
    # Establish a file name for speciesDict
    filename = outDir + species + "_CONUS_HabModel_2001v1.json"
    
    # Get python tuple (like a list) of regional model codes
    modelCodes = [x for x in gp.gapmodeling.ModelCodes(species, 
                                                 publishedOnly=True, 
                                                 conusOnly=True, 
                                                 migratory=False) if int(x[-1:]) in [1,2,3,4,5,6]]
    
    # Get the models as dictionaries, add to a dictionary of models, then add that
    # to the species level dictionary
    models = {}
    for model in modelCodes:
        modelDict = gp.gapmodeling.ModelAsDictionary(model, ecolSystem = "both")
        modelDict["intElevMax"] = 100 # !!!!!!!!!!!!!!!!!  REMOVE
        modelDict["intElevMin"] = 105 # !!!!!!!!!!!!!!!!!  REMOVE
        models[model] = modelDict
    speciesDict["models"] = models
    
    # Save species model dictionary as json object
    outfile = open(filename, "w")
    json.dump(speciesDict, outfile)
display(speciesDict)

rRNSNx
1 of 2
bEASOx
2 of 2


{'doi': '???????',
 'habitat_description': u'The Eastern Screech-Owl occurs east of the Rocky Mountains, where it is a permanent resident of both rural and urban habitats from south of the Canadian boreal forest to near the Tropic of Cancer in Mexico. This species nests in tree cavities in wooded environments below about 1,500 meters, regardless of habitat, occupying lowland forests to mountainside woodlands, both deciduous and evergreen. Because it readily habituates to people, this species sometimes nests in human-made cavities such as bird boxes. It is often the most common or only avian predator in wooded suburban and urban habitats.Generally, habitat consists of tree-dominated landscapes of most types, spanning age classes early successional to mature and elevations from mountain slopes to river valleys below 1500 meters. Forested environments include northern and subtropical deciduous and mixed forest, riparian deciduous and southern pine (Pinus) forest, southwestern oak (Quercus

In [26]:
print(speciesDict.keys())

['doi', 'input_layers', 'models', 'references', 'habitat_description', 'taxonomic']


# What's the next step to getting these onto ScienceBase?  sb.upload_file_to_item(item=?, outfile=?, scrape_file=?)??????  I'm not sure how that works.

